## Needs Work

Home Depot.com fronts with Akamai and they refuse bot traffic with an HTTP response like (yes, complete with the dangling `<P>`)
```
<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>

You don't have permission to access "http://www.homedepot.com/mcc-cart/v2/info/storefulfillment?" on this server.<P>
Reference #...
</BODY>
</HTML>
```

In this past this did not use to be the case. A bare curl request would always return. Setting a real User Agent string doesn't bypass this, you need to send back real cookies.

Therefore this code no longer functions as-is.

In [2]:
import json, urllib
from twilio.rest import Client

In [3]:
# Get the Item ID from the end of the product page URL
# For example: https://www.homedepot.com/p/Forum-Novelties-Clown-Around-Town-Child-Costume-F62198-M/204447677
item_id = '204447677'

# Your Account Sid and Auth Token from twilio.com/console
account_sid = 'AC...'
auth_token = '...'
twilio_from_number = '+1585475....'
sms_destination = '+1585475....'

client = Client(account_sid, auth_token)

In [4]:
def getQtys(store='1861'):
    url = 'https://www.homedepot.com/mcc-cart/v2/info/storefulfillment?itemId=' + item_id + '&keyword=' + store

    raw_resp = urllib.request.urlopen(url).read()
    resp = json.loads(raw_resp.decode())

    qty_now = resp['storeFulfillment']['storeFulfillmentDetails']['primaryStore']['fulfillmentOptions']['buyOnlinePickupInStore']['inventory']['onHandQuantity']
    qty_maybe = resp['storeFulfillment']['storeFulfillmentDetails']['primaryStore']['fulfillmentOptions']['buyOnlinePickupInStore']['inventory']['expectedQuantityAvailable']
    
    return [qty_now, qty_maybe]

In [5]:
def sendSMS(msg):
    message = client.messages \
                .create(
                     body=msg,
                     from_=twilio_from_number,
                     to=sms_destination
                 )

In [8]:
# The store number is shown when you select your local store
# For example: San Jose (ge) #1861, or East Palo Alto #6603

# Palo Alto store
if getQtys('6603') != [0,0]:
    sendSMS('Clowns in stock @ EPA')

# San Jose default store
if getQtys() != [0,0]:
    sendSMS('Clowns in stock @ San Jose')

## For running in GCP Cloud Function/Lambda

Note that Akamai denies GCP IPs with prejudice :) so this won't work without an Akamai bypass.

Create an HTTP triggered Cloud Function with source:
```python
import json, os, urllib
from twilio.rest import Client

item_id = '204447677'

account_sid = os.environ['TW_ACC']
auth_token = os.environ['TW_AUTH']
twilio_from_number = os.environ['FROM']
sms_destination = os.environ['TO']

client = Client(account_sid, auth_token)

def getQtys(store='1861'):
    url = 'https://www.homedepot.com/mcc-cart/v2/info/storefulfillment?itemId=' + item_id + '&keyword=' + store

    raw_resp = urllib.request.urlopen(url).read()
    resp = json.loads(raw_resp.decode())

    qty_now = resp['storeFulfillment']['storeFulfillmentDetails']['primaryStore']['fulfillmentOptions']['buyOnlinePickupInStore']['inventory']['onHandQuantity']
    qty_maybe = resp['storeFulfillment']['storeFulfillmentDetails']['primaryStore']['fulfillmentOptions']['buyOnlinePickupInStore']['inventory']['expectedQuantityAvailable']
    
    return [qty_now, qty_maybe]

def sendSMS(msg):
    message = client.messages \
                .create(
                     body=msg,
                     from_=twilio_from_number,
                     to=sms_destination
                 )

def checkStock(request):
    msg = ''
    
    if getQtys('6603') != [0,0]:
        sendSMS('Clowns in stock @ EPA')
        msg = 'In stock'

    if getQtys() != [0,0]:
        sendSMS('Clowns in stock @ San Jose')
        msg = 'In stock'
        
    return msg
```

And requirements.txt:
```
# Function dependencies, for example:
# package>=version
twilio==6.25.2
urllib3==1.24.1
```

Lang: Python 3.7
Memory: 128 MB
Function: checkStock
Max invocations: 1

Set env vars TW_ACC, TW_AUTH, FROM, and TO.

Create a Cloud Scheduler, with frequency `0 8-22 * * 1-5` (hourly every weekday between 8a-10p) making an HTTP GET to the Cloud Function trigger. Or use https://crontab.guru to create other timings.